In [ ]:
from utils.utils import compare_sinr_user_mean,compare_sinr_per_element, analyze_and_plot_best_episode, plot_metrics_vs_user,custom_save_plot_metrics_vs_user

# Define folders
import os
graphs_folder = "graphs_folder"
data_csv_folder = "data_csv_folder"

# Create folders if they don't exist
os.makedirs(graphs_folder, exist_ok=True)
os.makedirs(data_csv_folder, exist_ok=True)
saved_folder = "saved_data"

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:

# # To compare element-wise SINR:
# compare_sinr_per_element(saved_folder)

# To compare user-mean SINR:
best_episode_index = compare_sinr_user_mean(saved_folder)




In [ ]:
# Load baseline data
Thrpt_baseline = np.load(f'{saved_folder}/Baseline_Thrpt.npy')     # (20, 61)
SINR_baseline = np.load(f'{saved_folder}/Baseline_SINR.npy')       # (20, 61)
Intf_baseline = np.load(f'{saved_folder}/Baseline_Intf.npy')       # (20, 61)
SE_baseline = np.load(f'{saved_folder}/Baseline_SE.npy')       # (20, 61)

# Load best episode data
Thrpt_best = np.load(f'{saved_folder}/Episode_{best_episode_index}_Thrpt.npy')
SINR_best = np.load(f'{saved_folder}/Episode_{best_episode_index}_SINR.npy')
Intf_best = np.load(f'{saved_folder}/Episode_{best_episode_index}_Intf.npy')
SE_best = np.load(f'{saved_folder}/Episode_{best_episode_index}_SE.npy')  # Newly added

# Compute mean across timesteps (axis=1) for each user
mean_thrpt_baseline = np.mean(Thrpt_baseline, axis=1)
mean_thrpt_best = np.mean(Thrpt_best, axis=1)

mean_sinr_baseline = np.mean(SINR_baseline, axis=1)
mean_sinr_best = np.mean(SINR_best, axis=1)

mean_intf_baseline = np.mean(Intf_baseline, axis=1)
mean_intf_best = np.mean(Intf_best, axis=1)



users = np.arange(1, 21)  # User indices (1 to 20)




# print("mean_sinr_baseline :",mean_sinr_baseline)
# print("mean_sinr_best :",mean_sinr_best)

# print("mean_thrpt_baseline :",mean_thrpt_baseline)
# print("mean_thrpt_best :",mean_thrpt_best)


print("SE_baseline :",SE_baseline.shape)
print("SE_best :",SE_best.shape)




# Transposing to (121, 1)
SE_baseline_transposed = SE_baseline.T
SE_best_transposed = SE_best.T

# Check the result
print("Transposed SE_baseline shape:", SE_baseline_transposed.shape)
print("Transposed SE_best shape:", SE_best_transposed.shape)

# Console output
print(f"✅ Episode Index: {best_episode_index} | Mean Throughput (first 10 users): {np.mean(mean_thrpt_best[:10]):.4f} Mbps")

# Create figure with 2x2 subplots
plt.figure(figsize=(18, 10))

# 1. Throughput
plt.subplot(2, 2, 1)
plt.plot(users, mean_thrpt_baseline, label='Baseline', marker='o',color ='r')
plt.plot(users, mean_thrpt_best, label=f'Episode {best_episode_index}', marker='x',color ='b')
plt.title('Mean Throughput per User')
plt.xlabel('User Index')
plt.ylabel('Throughput (Mbps)')
plt.legend()
plt.grid(True)

# 2. SINR
plt.subplot(2, 2, 2)
plt.plot(users, mean_sinr_baseline, label='Baseline', marker='o',color ='r')
plt.plot(users, mean_sinr_best, label=f'Episode {best_episode_index}', marker='x',color ='b')
plt.title('Mean SINR per User')
plt.xlabel('User Index')
plt.ylabel('SINR (dB)')
plt.legend()
plt.grid(True)

# 3. Interference
plt.subplot(2, 2, 3)
plt.plot(users, mean_intf_baseline, label='Baseline', marker='o',color ='r')
plt.plot(users, mean_intf_best, label=f'Episode {best_episode_index}', marker='x',color ='b')
plt.title('Mean Interference per User')
plt.xlabel('User Index')
plt.ylabel('Interference (dB or mW)')
plt.legend()
plt.grid(True)

# 4. Spectral Efficiency (SE)
plt.subplot(2, 2, 4)
plt.plot( np.arange(1, 122), SE_baseline_transposed, label=f'Baseline {best_episode_index}', marker='o',color ='r')
plt.plot( np.arange(1, 122), SE_best_transposed, label=f'SE Episode {best_episode_index}', marker='x', color ='b')
plt.title('Mean Spectral Efficiency per User')
plt.xlabel('User Index')
plt.ylabel('SE (bits/s/Hz)')
plt.legend()
plt.grid(True)

# Save and display
plt.tight_layout()
filename_base = os.path.join(graphs_folder, f'Metrics_vs_User_Episode_{best_episode_index}')
plt.savefig(f'{filename_base}.png', dpi=300)
plt.savefig(f'{filename_base}.pdf', dpi=300)
plt.savefig(f'{filename_base}.eps', dpi=300, format='eps')
print(f"✅ Saved plots to: {filename_base}.{{png, pdf, eps}}")

plt.show()
plt.close()

In [ ]:
plot_metrics_vs_user(saved_folder,best_episode_index,graphs_folder)

In [ ]:
analyze_and_plot_best_episode(saved_folder,best_episode_index,graphs_folder,data_csv_folder)

In [ ]:
import pandas as pd
import os

# Assuming 'data_folder' is defined elsewhere in your code, e.g.:
# data_folder = 'path/to/your/data' 

leo_throughput_file = os.path.join(data_csv_folder, 'LEO_Throughput_data.csv')
geo_throughput_file = os.path.join(data_csv_folder, 'GEO_Throughput_data.csv')
leo_sinr_file = os.path.join(data_csv_folder, 'LEO_SINR_data.csv')
geo_sinr_file = os.path.join(data_csv_folder, 'GEO_SINR_data.csv')
leo_se_file = os.path.join(data_csv_folder, 'LEO_SE_data.csv')
geo_se_file = os.path.join(data_csv_folder, 'GEO_SE_data.csv')
all_se_file = os.path.join(data_csv_folder, 'all_SE_data.csv')

# Read the data files and rename columns
leo_throughput = pd.read_csv(leo_throughput_file).rename(columns={
    'Random Allocation': 'leo_thrpt_baseline',
    'Advanced DSA (A2C)': 'leo_thrpt_a2c'
})
geo_throughput = pd.read_csv(geo_throughput_file).rename(columns={
    'Random Allocation': 'geo_thrpt_baseline',
    'Advanced DSA (A2C)': 'geo_thrpt_a2c'
})
leo_sinr = pd.read_csv(leo_sinr_file).rename(columns={
    'Random Allocation': 'leo_sinr_baseline',
    'Advanced DSA (A2C)': 'leo_sinr_a2c'
})
geo_sinr = pd.read_csv(geo_sinr_file).rename(columns={
    'Random Allocation': 'geo_sinr_baseline',
    'Advanced DSA (A2C)': 'geo_sinr_a2c'
})
leo_se = pd.read_csv(leo_se_file).rename(columns={
    'Random Allocation': 'leo_se_baseline',
    'Advanced DSA (A2C)': 'leo_se_a2c'
})
geo_se = pd.read_csv(geo_se_file).rename(columns={
    'Random Allocation': 'geo_se_baseline',
    'Advanced DSA (A2C)': 'geo_se_a2c'
})
all_se = pd.read_csv(all_se_file).rename(columns={
    'Random Allocation': 'all_se_baseline',
    'Advanced DSA (A2C)': 'all_se_a2c'
})

# Start with one dataframe and merge others
combined_df = leo_throughput

# Merge the dataframes. Assuming 'Time (s)' is the common column to merge on.
# If the 'Time (s)' column is not consistent across all files, you might need
# to adjust the merging strategy (e.g., outer join, or handling missing times).

combined_df = pd.merge(combined_df, geo_throughput, on='Time (s)', how='outer')
combined_df = pd.merge(combined_df, leo_sinr, on='Time (s)', how='outer')
combined_df = pd.merge(combined_df, geo_sinr, on='Time (s)', how='outer')
combined_df = pd.merge(combined_df, leo_se, on='Time (s)', how='outer')
combined_df = pd.merge(combined_df, geo_se, on='Time (s)', how='outer')
combined_df = pd.merge(combined_df, all_se, on='Time (s)', how='outer')

# Save the combined dataframe to a new CSV file
output_combined_file = os.path.join(data_csv_folder, 'merged_leo_geo_data.csv')
combined_df.to_csv(output_combined_file, index=False)

print(f"All data combined and saved to '{output_combined_file}'")
# print(combined_df.head()) # Display the first few rows of the combined dataframe
print(combined_df.columns) # Display the first few rows of the combined dataframe